In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV # GridSearchCV added back
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
# New models to import
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier # Added XGBoost

# Existing models for time series
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.datatypes._panel._convert import from_2d_array_to_nested

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchsummary import summary # Make sure this is installed: pip install torchsummary

# Feature extraction imports
from scipy.fftpack import fft
import pywt
import pmdarima as pm # For auto_arima
from prophet import Prophet # For Prophet
#from prophet.utilities import stan_init # For Prophet with PyStan 3+
import logging # To suppress Prophet logs

# Suppress excessive logging from Prophet and CmdStanPy
logging.getLogger('prophet').setLevel(logging.ERROR)
logging.getLogger('cmdstanpy').setLevel(logging.ERROR)


#==== Load data ====
print("Loading data...")
train_df = pd.read_csv('Sleep Train 5000.csv', header=None)
test_df = pd.read_csv('Sleep Test 1000.csv', header=None)
print("Data loaded.")

#==== EDA and Visualization (using full raw training data before splits) ====

X_train_full_viz = train_df.iloc[:, 1:].values
y_train_full_viz = train_df.iloc[:, 0].values
def explore_data(df, name="Dataset"): ...
train_classes_counts = explore_data(train_df, "Training Data")
def plot_examples(X, y, n_examples=2): ...
plot_examples(X_train_full_viz, y_train_full_viz)

# --- Custom Feature Extraction Functions ---
def extract_stats(X_data):
    features = []
    for signal in X_data:
        mean = np.mean(signal); std = np.std(signal); var = np.var(signal)
        mini = np.min(signal); maxi = np.max(signal); rng = maxi - mini
        median = np.median(signal); skew = 0; kurtosis = 0
        if std > 1e-6: # Avoid division by zero for flat signals
            skew = np.mean(((signal - mean) / std) ** 3)
            kurtosis = np.mean(((signal - mean) / std) ** 4) - 3
        else: # Handle flat signals
            skew = 0
            kurtosis = -3 # Kurtosis of a constant is -3 (excess kurtosis)
            
        zero_crossings = np.sum(np.diff(np.signbit(signal - np.mean(signal))))
        # Peak finding can be sensitive; this is a simple version
        diff_signal = np.diff(signal)
        peaks = np.sum(np.diff(np.signbit(diff_signal)) < 0  & (diff_signal[:-1] > 0)) # Ensure it's a peak

        diffs = np.diff(signal)
        mean_diff = np.mean(diffs) if len(diffs) > 0 else 0
        std_diff = np.std(diffs) if len(diffs) > 0 else 0
        energy = np.sum(signal ** 2); power = energy / len(signal) if len(signal) > 0 else 0
        q25 = np.percentile(signal, 25); q75 = np.percentile(signal, 75); iqr = q75 - q25
        feat = [mean, std, var, mini, maxi, rng, median, skew, kurtosis,
                zero_crossings, peaks, mean_diff, std_diff, energy, power, q25, q75, iqr]
        features.append(feat)
    return np.array(features)

def extract_fft_features(X_data, n_coeffs=50):
    features = []
    for signal in X_data:
        fft_values_complex = fft(signal)
        fft_values = np.abs(fft_values_complex)
        
        # Ensure n_coeffs is not larger than half the FFT length
        eff_n_coeffs = min(n_coeffs, len(fft_values) // 2)
        dominant_freqs = fft_values[:eff_n_coeffs]
        if len(dominant_freqs) < n_coeffs: # Pad if signal was too short
            dominant_freqs = np.pad(dominant_freqs, (0, n_coeffs - len(dominant_freqs)), 'constant')

        total_power = np.sum(fft_values ** 2)
        total_power = total_power if total_power > 1e-6 else 1.0 # Avoid division by zero

        n_bands = 5
        # Ensure band_size is at least 1
        band_size = max(1, len(fft_values) // (2 * n_bands)) # Use only first half of FFT spectrum for bands
        band_powers = []
        for i in range(n_bands):
            start = i * band_size
            end = min((i + 1) * band_size, len(fft_values) // 2)
            if start >= end: # if signal too short, band might be empty
                band_powers.append(0)
                continue
            band_power = np.sum(fft_values[start:end] ** 2) / total_power
            band_powers.append(band_power)
        
        fft_mean = np.mean(fft_values[:len(fft_values)//2]) # Mean of magnitude spectrum (positive freqs)
        fft_std = np.std(fft_values[:len(fft_values)//2])
        fft_max = np.max(fft_values[:len(fft_values)//2])
        fft_peak_idx = np.argmax(fft_values[:len(fft_values)//2]) if len(fft_values) > 0 else 0
        
        feat = np.concatenate([dominant_freqs, band_powers, [fft_mean, fft_std, fft_max, fft_peak_idx]])
        features.append(feat)
    return np.array(features)

def extract_wavelet_features(X_data, wavelet='db4', level=4):
    features = []
    # Calculate expected length once
    # wavedec returns level+1 arrays of coeffs. For each, 3 stats (mean, std, energy).
    expected_feature_length = (level + 1) * 3

    for signal in X_data:
        wavelet_features_single = []
        try:
            # Ensure signal is long enough for the decomposition level
            min_len = pywt.dwt_max_level(len(signal), pywt.Wavelet(wavelet))
            actual_level = min(level, min_len) # Adjust level if signal is too short
            
            if actual_level < 1 : # Cannot decompose
                 coeffs = [] # or handle as an error / default features
            else:
                coeffs = pywt.wavedec(signal, wavelet, level=actual_level)

            for i in range(level + 1): # Iterate up to the original requested level
                if i < len(coeffs) and len(coeffs[i]) > 0:
                    coef = coeffs[i]
                    wavelet_features_single.extend([np.mean(coef), np.std(coef), np.sum(coef ** 2)])
                else:
                    # If coefficient array is missing (due to adjusted level) or empty, pad with zeros
                    wavelet_features_single.extend([0.0, 0.0, 0.0])
        
        except ValueError: # Catch other pywt errors
            wavelet_features_single = [0.0] * expected_feature_length
        
        # Ensure the feature vector has the expected length
        if len(wavelet_features_single) < expected_feature_length:
            wavelet_features_single.extend([0.0] * (expected_feature_length - len(wavelet_features_single)))
        elif len(wavelet_features_single) > expected_feature_length:
            wavelet_features_single = wavelet_features_single[:expected_feature_length]
            
        features.append(wavelet_features_single)
    return np.array(features)

def extract_arima_features(X_data):
    print("Extracting ARIMA features (this may take a very long time)...")
    features = []
    # Define a fixed number of features to extract, e.g., p, d, q, aic, bic, sigma2
    # If seasonal: P, D, Q, S. For non-seasonal: 3 params for order, 3 for seasonal_order (0,0,0,0), AIC, BIC, sigma2
    num_arima_feats = 3 + 1 + 1 + 1 # p,d,q, aic, bic, sigma2
    
    for i, signal in enumerate(X_data):
        if (i + 1) % 100 == 0:
            print(f"  Processed {i+1}/{len(X_data)} signals for ARIMA features.")
        try:
            # auto_arima can be very slow.
            # Consider simplifying: stepwise=True, trace=False, error_action='ignore',
            # suppress_warnings=True, seasonal=False (if no strong seasonality expected)
            model = pm.auto_arima(signal,
                                  start_p=1, start_q=1,
                                  max_p=3, max_q=3, # Keep p,q low to speed up
                                  d=None,           # Let auto_arima find d
                                  seasonal=False,   # Assuming non-seasonal for general segments
                                  stepwise=True,    # Faster
                                  suppress_warnings=True,
                                  error_action='ignore', # Ignore errors and return a simpler model
                                  trace=False)
            
            if model is not None and hasattr(model, 'order'):
                p, d, q = model.order
                aic = model.aic() if hasattr(model, 'aic') else np.nan
                bic = model.bic() if hasattr(model, 'bic') else np.nan
                sigma2 = model.params().get('sigma2', np.nan) # Get sigma2 if available
                feat = [p, d, q, aic, bic, sigma2]
            else: # Model fitting failed or returned None
                feat = [np.nan] * num_arima_feats

        except Exception as e: # Catch any other errors during ARIMA fitting
            # print(f"ARIMA error for signal {i}: {e}")
            feat = [np.nan] * num_arima_feats
        features.append(feat)
    
    arima_features_array = np.array(features)
    # Impute NaNs that may have arisen from fitting issues or missing attributes
    # Using column means for imputation; ensure there's at least one valid value per column
    if arima_features_array.shape[0] > 0:
        col_means = np.nanmean(arima_features_array, axis=0)
        # If a whole column is NaN, nanmean returns NaN. Replace these with 0.
        col_means = np.where(np.isnan(col_means), 0, col_means)
        for j in range(arima_features_array.shape[1]):
            col_data = arima_features_array[:, j]
            arima_features_array[np.isnan(col_data), j] = col_means[j]
            
    return arima_features_array


def extract_prophet_features(X_data):
    print("Extracting Prophet features (this may also take a long time)...")
    features = []
    # Define fixed features: trend slope, number of changepoints, maybe forecast error std
    num_prophet_feats = 3 # Example: growth rate, n_changepoints, trend_std_error
    
    for i, signal in enumerate(X_data):
        if (i + 1) % 100 == 0:
            print(f"  Processed {i+1}/{len(X_data)} signals for Prophet features.")
        
        df_prophet = pd.DataFrame({
            'ds': pd.to_datetime(pd.RangeIndex(start=0, stop=len(signal), step=1), unit='s'),
            'y': signal
        })
        
        try:
            # Suppress Stan initialization messages
            m = Prophet(yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False,
                        # uncertainty_samples=False, # Faster, but no error bands for trend
                        changepoint_range=0.8) # Default
            
            # For PyStan 3+, Prophet might need explicit stan_init
            # m.stan_backend.stan_fit = stan_init(m.model) # Uncomment if you see Stan init issues

            m.fit(df_prophet, iter=500) # Reduce iterations for speed if needed

            # Extract features
            growth_rate = m.params['delta'][0][0] if 'delta' in m.params and m.params['delta'].size > 0 else 0.0
            num_changepoints = len(m.changepoints)
            
            # Trend standard error (if uncertainty_samples > 0 or was True)
            # This requires forecast to be made.
            # future = m.make_future_dataframe(periods=0)
            # forecast = m.predict(future)
            # trend_std_error = forecast['trend_upper'].iloc[-1] - forecast['trend_lower'].iloc[-1] if 'trend_upper' in forecast else 0.0
            # For simplicity now, let's use a placeholder for the third feature.
            # One option is the initial trend value (offset k)
            initial_trend = m.params['k'][0][0] if 'k' in m.params and m.params['k'].size > 0 else 0.0

            feat = [growth_rate, num_changepoints, initial_trend]

        except Exception as e:
            # print(f"Prophet error for signal {i}: {e}")
            feat = [0.0] * num_prophet_feats # Default values on error
        features.append(feat)
    return np.array(features)


#==== Data Preparation ====
X_all_train_raw = train_df.iloc[:, 1:].values
y_all_train_raw = train_df.iloc[:, 0].values

# Ensure test data has consistent number of features as training data before feature extraction
# (assuming test_df contains only features, no label column)
X_test_full_raw = test_df.values
if X_test_full_raw.shape[1] < X_all_train_raw.shape[1]:
    print(f"Padding test data from {X_test_full_raw.shape[1]} to {X_all_train_raw.shape[1]} features.")
    padding = np.zeros((X_test_full_raw.shape[0], X_all_train_raw.shape[1] - X_test_full_raw.shape[1]))
    X_test_full_raw = np.hstack((X_test_full_raw, padding))
elif X_test_full_raw.shape[1] > X_all_train_raw.shape[1]:
    print(f"Truncating test data from {X_test_full_raw.shape[1]} to {X_all_train_raw.shape[1]} features.")
    X_test_full_raw = X_test_full_raw[:, :X_all_train_raw.shape[1]]

le = LabelEncoder()
y_all_train_encoded = le.fit_transform(y_all_train_raw)
num_classes = len(le.classes_)

X_train_raw, X_val_raw, y_train, y_val = train_test_split(
    X_all_train_raw, y_all_train_encoded, test_size=0.2, random_state=42, stratify=y_all_train_encoded
)

# --- Feature Engineering ---
# Option to skip slow features for faster iteration:
RUN_ARIMA_FEATURES = False # SET TO TRUE TO RUN (VERY SLOW)
RUN_PROPHET_FEATURES = False # SET TO TRUE TO RUN (VERY SLOW)

print("Extracting statistical features...")
X_train_stats = extract_stats(X_train_raw)
X_val_stats = extract_stats(X_val_raw)
X_test_stats = extract_stats(X_test_full_raw)

print("Extracting FFT features...")
X_train_fft = extract_fft_features(X_train_raw)
X_val_fft = extract_fft_features(X_val_raw)
X_test_fft = extract_fft_features(X_test_full_raw)

print("Extracting Wavelet features...")
X_train_wave = extract_wavelet_features(X_train_raw)
X_val_wave = extract_wavelet_features(X_val_raw)
X_test_wave = extract_wavelet_features(X_test_full_raw)

# Initialize lists for feature sets
train_feature_sets = [X_train_stats, X_train_fft, X_train_wave]
val_feature_sets = [X_val_stats, X_val_fft, X_val_wave]
test_feature_sets = [X_test_stats, X_test_fft, X_test_wave]

if RUN_ARIMA_FEATURES:
    print("Extracting ARIMA features for TRAIN data...")
    X_train_arima = extract_arima_features(X_train_raw)
    train_feature_sets.append(X_train_arima)
    
    print("Extracting ARIMA features for VALIDATION data...")
    X_val_arima = extract_arima_features(X_val_raw)
    val_feature_sets.append(X_val_arima)

    print("Extracting ARIMA features for TEST data...")
    X_test_arima = extract_arima_features(X_test_full_raw)
    test_feature_sets.append(X_test_arima)
else:
    print("Skipping ARIMA feature extraction.")

if RUN_PROPHET_FEATURES:
    print("Extracting Prophet features for TRAIN data...")
    X_train_prophet = extract_prophet_features(X_train_raw)
    train_feature_sets.append(X_train_prophet)

    print("Extracting Prophet features for VALIDATION data...")
    X_val_prophet = extract_prophet_features(X_val_raw)
    val_feature_sets.append(X_val_prophet)

    print("Extracting Prophet features for TEST data...")
    X_test_prophet = extract_prophet_features(X_test_full_raw)
    test_feature_sets.append(X_test_prophet)
else:
    print("Skipping Prophet feature extraction.")

X_train_engineered = np.hstack(train_feature_sets)
X_val_engineered = np.hstack(val_feature_sets)
X_test_engineered = np.hstack(test_feature_sets)

scaler_eng = StandardScaler()
X_train_eng_scaled = scaler_eng.fit_transform(X_train_engineered)
X_val_eng_scaled = scaler_eng.transform(X_val_engineered)
X_test_eng_scaled = scaler_eng.transform(X_test_engineered)
print(f"Shape of engineered train features: {X_train_eng_scaled.shape}")

# Prepare data for sktime and PyTorch (raw time series)
scaler_ts = StandardScaler() # Use a separate scaler for raw time series
X_train_ts_scaled = scaler_ts.fit_transform(X_train_raw)
X_val_ts_scaled = scaler_ts.transform(X_val_raw)
X_test_ts_scaled = scaler_ts.transform(X_test_full_raw)

X_train_nested = from_2d_array_to_nested(X_train_ts_scaled)
X_val_nested = from_2d_array_to_nested(X_val_ts_scaled)
X_test_nested = from_2d_array_to_nested(X_test_ts_scaled)

#==== Train Models ====
models = {}
val_accuracies = {}

# --- Traditional Models using Engineered Features ---
print("\nTraining Logistic Regression on engineered features...")
log_reg = LogisticRegression(random_state=42, max_iter=2000, solver='liblinear', class_weight='balanced')
log_reg.fit(X_train_eng_scaled, y_train)
models['Logistic Regression (Eng. Feat.)'] = log_reg

print("Training Random Forest on engineered features (with GridSearchCV)...")
# Example of GridSearchCV for RandomForest
param_grid_rf = {
    'n_estimators': [100, 150, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'class_weight': ['balanced', 'balanced_subsample']
}
# Reduce CV folds for speed during development if needed, e.g., cv=3
# n_jobs=-1 uses all available cores
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42, n_jobs=-1), 
                       param_grid_rf, cv=3, scoring='accuracy', verbose=1)
rf_grid.fit(X_train_eng_scaled, y_train)
print(f"Best RF params: {rf_grid.best_params_}")
rf_clf = rf_grid.best_estimator_
models['Random Forest (Eng. Feat.)'] = rf_clf


print("Training SVM on engineered features...")
# SVM can be slow. Consider GridSearchCV with a smaller parameter space or fewer C values.
# For now, using your previous parameters with class_weight.
svm_clf = SVC(random_state=42, C=1.0, kernel='rbf', class_weight='balanced', probability=False)
svm_clf.fit(X_train_eng_scaled, y_train)
models['SVM (Eng. Feat.)'] = svm_clf

print("Training Decision Tree on engineered features...")
dt_clf = DecisionTreeClassifier(random_state=42, class_weight='balanced', max_depth=10)
dt_clf.fit(X_train_eng_scaled, y_train)
models['Decision Tree (Eng. Feat.)'] = dt_clf

print("Training KNN on engineered features...")
knn_clf = KNeighborsClassifier(n_neighbors=7, n_jobs=-1) # Consider tuning n_neighbors with GridSearchCV
knn_clf.fit(X_train_eng_scaled, y_train)
models['KNN (Eng. Feat.)'] = knn_clf

print("Training AdaBoost on engineered features...")
ada_clf = AdaBoostClassifier(n_estimators=100, random_state=42, learning_rate=0.1)
ada_clf.fit(X_train_eng_scaled, y_train)
models['AdaBoost (Eng. Feat.)'] = ada_clf

print("Training XGBoost on engineered features...")
xgb_clf = XGBClassifier(random_state=42, n_estimators=150, use_label_encoder=False, eval_metric='mlogloss',
                        learning_rate=0.1, max_depth=7, # Example parameters, tune these
                        scale_pos_weight=None) # For imbalanced: compute weights or use 'balanced' logic
# For multiclass with XGBoost, it handles labels 0..num_class-1 internally. y_train is already encoded.
# If classes are imbalanced, consider `scale_pos_weight` (for binary) or compute sample weights.
# For now, relying on the inherent robustness of XGBoost.
xgb_clf.fit(X_train_eng_scaled, y_train)
models['XGBoost (Eng. Feat.)'] = xgb_clf

# --- Models using Scaled Time Series Data ---
print("Training TimeSeries Forest on scaled TS data...")
ts_forest = TimeSeriesForestClassifier(n_estimators=150, random_state=42, n_jobs=-1) # Consider tuning
ts_forest.fit(X_train_nested, y_train)
models['TimeSeries Forest (Raw TS)'] = ts_forest

# --- Evaluate on validation (before CNN) ---
print("\nValidation Accuracies (Traditional Models & TSF):")
for name, model in models.items():
    if name == 'TimeSeries Forest (Raw TS)':
        val_preds = model.predict(X_val_nested)
    else: # Models trained on engineered features
        val_preds = model.predict(X_val_eng_scaled)
    acc = accuracy_score(y_val, val_preds)
    val_accuracies[name] = acc
    print(f"{name}: {acc:.4f}")
    # print(classification_report(y_val, val_preds, target_names=[str(c) for c in le.classes_]))


# --- 1D-CNN Model (PyTorch) ---
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("\nUsing GPU for PyTorch")
else:
    device = torch.device("cpu")
    print("\nUsing CPU for PyTorch")

X_train_cnn = torch.tensor(X_train_ts_scaled.reshape(-1, 1, X_train_ts_scaled.shape[1]), dtype=torch.float32)
y_train_cnn = torch.tensor(y_train, dtype=torch.long)
X_val_cnn = torch.tensor(X_val_ts_scaled.reshape(-1, 1, X_val_ts_scaled.shape[1]), dtype=torch.float32)
y_val_cnn = torch.tensor(y_val, dtype=torch.long)
X_test_cnn_tensor = torch.tensor(X_test_ts_scaled.reshape(-1, 1, X_test_ts_scaled.shape[1]), dtype=torch.float32)

train_dataset = TensorDataset(X_train_cnn, y_train_cnn)
val_dataset = TensorDataset(X_val_cnn, y_val_cnn)
test_dataset_cnn = TensorDataset(X_test_cnn_tensor) # Note: This only has X, no y

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader_cnn = DataLoader(test_dataset_cnn, batch_size=64, shuffle=False)

class CNN1D(nn.Module):
    def __init__(self, num_features_in, num_classes_out): # Corrected: def __init__
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=7, padding=3)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=7, padding=3)
        self.bn2 = nn.BatchNorm1d(64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2)

        self.conv3 = nn.Conv1d(64, 128, kernel_size=7, padding=3)
        self.bn3 = nn.BatchNorm1d(128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool1d(kernel_size=2)
        
        # Dynamic flattening size calculation
        with torch.no_grad():
            dummy_input_channel = 1 # Matches in_channels of self.conv1
            dummy = torch.randn(1, dummy_input_channel, num_features_in)
            # Pass through convolutional and pooling layers
            dummy = self.pool1(self.relu1(self.bn1(self.conv1(dummy))))
            dummy = self.pool2(self.relu2(self.bn2(self.conv2(dummy))))
            dummy = self.pool3(self.relu3(self.bn3(self.conv3(dummy))))
            flattened_size = dummy.shape[1] * dummy.shape[2]
            # print(f"Calculated flattened size for CNN: {flattened_size}")

        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(flattened_size, 256)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(256, num_classes_out)

    def forward(self, x):
        x = self.pool1(self.relu1(self.bn1(self.conv1(x))))
        x = self.pool2(self.relu2(self.bn2(self.conv2(x))))
        x = self.pool3(self.relu3(self.bn3(self.conv3(x))))
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.relu4(self.fc1(x))
        x = self.fc2(x)
        return x

num_input_features_cnn = X_train_ts_scaled.shape[1]
cnn_model = CNN1D(num_features_in=num_input_features_cnn, num_classes_out=num_classes).to(device)
try:
    print(summary(cnn_model, input_size=(1, num_input_features_cnn)))
except Exception as e:
    print(f"Error generating model summary (likely due to input shape issues if not on GPU): {e}")
    print(f"Make sure model is on the same device as dummy input for summary if not using CPU.")


criterion = nn.CrossEntropyLoss() # Add label_smoothing=0.1 for regularization if desired
optimizer = optim.Adam(cnn_model.parameters(), lr=0.0005, weight_decay=1e-4) # Try AdamW too
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)

num_epochs_cnn = 60 # Or more, with early stopping
best_val_acc_cnn = 0.0
patience_cnn = 15 # Original patience
patience_counter_cnn = 0
 
print("\nTraining 1D-CNN...")
for epoch in range(num_epochs_cnn):
    cnn_model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = cnn_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)

    cnn_model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = cnn_model(inputs)
            loss_v = criterion(outputs, labels) # Use different var name for val loss item
            val_loss += loss_v.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_val_loss = val_loss / len(val_loader.dataset)
    epoch_val_acc = correct / total
    print(f"Epoch {epoch+1}/{num_epochs_cnn}, Train Loss: {epoch_loss:.4f}, Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}")

    scheduler.step(epoch_val_loss)

    if epoch_val_acc > best_val_acc_cnn:
        best_val_acc_cnn = epoch_val_acc
        torch.save(cnn_model.state_dict(), 'best_cnn_1d_model.pth')
        print(f"New best CNN val acc: {best_val_acc_cnn:.4f}. Model saved.")
        patience_counter_cnn = 0
    else:
        patience_counter_cnn += 1
    
    if patience_counter_cnn >= patience_cnn:
        print(f"CNN Early stopping at epoch {epoch+1}.")
        break

# Load best model for final evaluation and prediction
if best_val_acc_cnn > 0: # Check if any model was saved
    print(f"Loading best CNN model with val acc: {best_val_acc_cnn:.4f}")
    cnn_model.load_state_dict(torch.load('best_cnn_1d_model.pth'))
else: # Should not happen if training ran at least one epoch
    print("No best CNN model saved, using last state (potential issue).")


models['1D-CNN (Raw TS)'] = cnn_model # Store the PyTorch model itself
val_accuracies['1D-CNN (Raw TS)'] = best_val_acc_cnn # Store its best validation accuracy
print(f"Best 1D-CNN Validation Accuracy after training: {best_val_acc_cnn:.4f}")


#==== Final Predictions on Test Set ====
print("\nPredicting on test set and saving results...")
all_predictions_df = pd.DataFrame() # To store all predictions if needed later

for name, model_object in models.items():
    filename_prefix = name.lower().replace(' (eng. feat.)', '_eng_feat') \
                                .replace(' (raw ts)', '_raw_ts') \
                                .replace(' ', '_').replace('(', '').replace(')', '')
    preds_original_labels = None # Initialize

    if name == 'TimeSeries Forest (Raw TS)':
        test_preds = model_object.predict(X_test_nested)
        preds_original_labels = le.inverse_transform(test_preds)
    elif name == '1D-CNN (Raw TS)':
        model_object.eval() # Ensure model is in evaluation mode
        cnn_test_preds_list = []
        with torch.no_grad():
            for inputs_batch in test_loader_cnn:
                inputs_tensor = inputs_batch[0].to(device)
                outputs = model_object(inputs_tensor)
                _, predicted = torch.max(outputs.data, 1)
                cnn_test_preds_list.extend(predicted.cpu().numpy())
        test_preds = np.array(cnn_test_preds_list)
        preds_original_labels = le.inverse_transform(test_preds)
    else: # Models trained on engineered features
        test_preds = model_object.predict(X_test_eng_scaled)
        preds_original_labels = le.inverse_transform(test_preds)

    if preds_original_labels is not None:
        # Store predictions in a common structure if you want to analyze them together later
        all_predictions_df[name] = preds_original_labels 
        
        # Save individual prediction files
        pd.DataFrame(preds_original_labels).to_csv(f"{filename_prefix}_predictions.csv", index=False, header=False)
        print(f"Saved: {filename_prefix}_predictions.csv")
    else:
        print(f"Could not generate predictions for {name}")

print("\n--- Final Validation Accuracies Summary ---")
# Sort by accuracy
sorted_val_accuracies = sorted(val_accuracies.items(), key=lambda item: item[1], reverse=True)
for model_name, acc in sorted_val_accuracies:
    print(f"{model_name}: {acc:.4f}")

best_model_name = sorted_val_accuracies[0][0] if sorted_val_accuracies else "N/A"
best_model_accuracy = sorted_val_accuracies[0][1] if sorted_val_accuracies else 0.0
print(f"\nBest performing model on validation set: {best_model_name} with accuracy: {best_model_accuracy:.4f}")

print("\nAll operations complete.")

Loading data...
Data loaded.
Extracting statistical features...
Extracting FFT features...
Extracting Wavelet features...
Skipping ARIMA feature extraction.
Skipping Prophet feature extraction.
Shape of engineered train features: (3999, 92)

Training Logistic Regression on engineered features...
Training Random Forest on engineered features (with GridSearchCV)...
Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best RF params: {'class_weight': 'balanced_subsample', 'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
Training SVM on engineered features...
Training Decision Tree on engineered features...
Training KNN on engineered features...
Training AdaBoost on engineered features...
Training XGBoost on engineered features...


C:\Users\Semhane\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [02:46:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Training TimeSeries Forest on scaled TS data...

Validation Accuracies (Traditional Models & TSF):
Logistic Regression (Eng. Feat.): 0.6370
Random Forest (Eng. Feat.): 0.6880
SVM (Eng. Feat.): 0.6650
Decision Tree (Eng. Feat.): 0.5730
KNN (Eng. Feat.): 0.6300
AdaBoost (Eng. Feat.): 0.5450
XGBoost (Eng. Feat.): 0.7010
TimeSeries Forest (Raw TS): 0.5660

Using CPU for PyTorch
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 32, 178]             256
       BatchNorm1d-2              [-1, 32, 178]              64
              ReLU-3              [-1, 32, 178]               0
         MaxPool1d-4               [-1, 32, 89]               0
            Conv1d-5               [-1, 64, 89]          14,400
       BatchNorm1d-6               [-1, 64, 89]             128
              ReLU-7               [-1, 64, 89]               0
         MaxPool1d-8               [-1, 64, 44

# Data # 
#### Training Data: ####
- Sleep Train 5000.csv (5000 samples, 179 columns: 1 label + 178 signal time points).
#### Test Data: ####
- Sleep Test 1000.csv (1000 samples, 178 signal time points).

# Methodology #
## 1. Data Preprocessing ##
- Labels were string-encoded and then numerically encoded using LabelEncoder.
- Data was split into training (80%) and validation (20%) sets, stratified by class.
#### Two main data preparation paths were followed: ####
##### Engineered Features: #####
- Raw signals were processed to extract statistical, FFT, and Wavelet features.
- ARIMA and Prophet features were available but disabled for this run due to computational cost.
These engineered features were then scaled using StandardScaler.
##### Raw Time Series: #####
- Raw signals were directly scaled using a separate StandardScaler for use with time-series specific models (TimeSeries Forest, 1D-CNN).

## 2. Feature Engineering ##
- **Statistical Features (18 features):**
Mean, std, var, min, max, range, median, skew, kurtosis, zero-crossings, peaks, mean/std of differences, energy, power, quartiles, IQR.
- **FFT Features (50 + 5 + 4 = 59 features):**
50 dominant frequency coefficients, 5 power band ratios, and mean/std/max/peak index of the FFT spectrum.
- **Wavelet Features ( (4+1)*3 = 15 features):**
Mean, std, and energy of approximation and detail coefficients from a 4-level 'db4' wavelet decomposition.
- **Total Engineered Features (without ARIMA/Prophet):**
18 + 59 + 15 = 92 features.

## 3. Model Training & Evaluation ##
A diverse set of models was trained and evaluated:
### Models on Engineered Features: ###
- Logistic Regression (with class balancing)
- Random Forest (with GridSearchCV for hyperparameter tuning: n_estimators, max_depth, min_samples_split/leaf, class_weight)
- Support Vector Machine (SVC with RBF kernel, class balancing)
- Decision Tree (with class balancing, max_depth=10)
- K-Nearest Neighbors (KNN, n_neighbors=7)
- AdaBoost Classifier
- XGBoost Classifier (initial run with default-ish parameters)

### Models on Scaled Raw Time Series: ###
- TimeSeries Forest Classifier (n_estimators=150)
- 1D Convolutional Neural Network (CNN) via PyTorch:
- **Architecture:** 3 Conv1D layers (kernels=7, padding=3, MaxPool1D after each), BatchNorm, ReLU, followed by Flatten, Dropout (0.5), and 2 Dense layers.
- **Optimizer:** Adam (lr=0.0005, weight_decay=1e-4).
- **Loss:** CrossEntropyLoss.
- **Training:** 60 epochs with early stopping (patience=15) and ReduceLROnPlateau scheduler based on validation loss.

## 4. Results (Based on Validation Set Performance) ##
- 1D-CNN (Raw TS): 0.7190
- XGBoost (Eng. Feat.): 0.7010
- Random Forest (Eng. Feat.): 0.6880
- SVM (Eng. Feat.): 0.6650
- Logistic Regression (Eng. Feat.): 0.6370
- KNN (Eng. Feat.): 0.6300
- Decision Tree (Eng. Feat.): 0.5730
- TimeSeries Forest (Raw TS): 0.5660
- AdaBoost (Eng. Feat.): 0.5450

- The 1D-CNN trained on raw scaled time series achieved the highest validation accuracy (0.7200).
- Among models using engineered features, XGBoost (0.7010) and Random Forest (0.6880) performed best.
- The initial TimeSeries Forest classifier performed less competitively on this dataset compared to the CNN and top engineered feature models.

## 5. Conclusions & Next Steps ##
The initial pipeline demonstrates the feasibility of classifying sleep stages, with the 1D-CNN showing the most promise. The engineered features also provide a solid foundation for traditional ensemble models.
Recommendations for Improvement (to reach ~0.8 accuracy target):
Systematic Hyperparameter Tuning:
- Expand GridSearchCV for XGBoost and Random Forest.
- Utilize Optuna (or similar) for more comprehensive CNN hyperparameter search (architecture, optimizer, regularization).
- Advanced Ensemble Methods: Explore stacking or weighted voting of the best performing models (e.g., CNN, XGBoost, RF).

In [9]:
import optuna

from sklearn.model_selection import StratifiedKFold # Ensure this is imported

# ==== XGBoost Training with GridSearchCV (Refined) ====
print("\nTraining XGBoost on engineered features (with EXPANDED GridSearchCV)...")

cv_stratified = StratifiedKFold(n_splits=3, shuffle=True, random_state=42) # 3 splits for speed, 5 for better estimate


param_grid_xgb = {
    'n_estimators': [200, 300, 400],          # Number of trees
    'learning_rate': [0.03, 0.05, 0.1],       # Step size shrinkage
    'max_depth': [5, 7, 9],                   # Max depth of a tree
    'subsample': [0.8, 0.9],                  # Subsample ratio of the training instance
    'colsample_bytree': [0.8, 0.9],           # Subsample ratio of columns when constructing each tree
    'gamma': [0, 0.1],                        # Minimum loss reduction required to make a further partition
    'min_child_weight': [1, 3]                # Minimum sum of instance weight (hessian) needed in a child
}

xgb_grid_search = GridSearchCV(
    XGBClassifier(random_state=42, eval_metric='mlogloss', use_label_encoder=False), # use_label_encoder=False for newer XGBoost
    param_grid_xgb,
    cv=cv_stratified,
    scoring='accuracy',
    verbose=2, # Set to 1 or 0 for less output
    n_jobs=-1  # Use all available cores
)

xgb_grid_search.fit(X_train_eng_scaled, y_train) # Make sure these are your scaled engineered features

print(f"Best XGBoost params: {xgb_grid_search.best_params_}")
print(f"Best XGBoost validation accuracy (from CV): {xgb_grid_search.best_score_:.4f}")

xgb_clf = xgb_grid_search.best_estimator_ # This is your tuned XGBoost
models['XGBoost (Eng. Feat.)'] = xgb_clf
# ==== 1D-CNN Model with Optuna Hyperparameter Tuning ====

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("\nUsing GPU for PyTorch")
else:
    device = torch.device("cpu")
    print("\nUsing CPU for PyTorch")

# Prepare data tensors (ensure these are defined: X_train_ts_scaled, y_train, X_val_ts_scaled, y_val)
X_train_cnn_opt = torch.tensor(X_train_ts_scaled.reshape(-1, 1, X_train_ts_scaled.shape[1]), dtype=torch.float32)
y_train_cnn_opt = torch.tensor(y_train, dtype=torch.long)
X_val_cnn_opt = torch.tensor(X_val_ts_scaled.reshape(-1, 1, X_val_ts_scaled.shape[1]), dtype=torch.float32)
y_val_cnn_opt = torch.tensor(y_val, dtype=torch.long)

# Global variable for input features, will be set before Optuna study
num_input_features_cnn_global = X_train_ts_scaled.shape[1]
num_classes_global = num_classes # num_classes should be defined from LabelEncoder

# Define the CNN1D class (as you have it, ensure __init__ is correct)
class CNN1D(nn.Module):
    def __init__(self, num_features_in, num_classes_out, n_filters_1, n_filters_2, n_filters_3, fc_units, dropout_rate):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=n_filters_1, kernel_size=7, padding=3)
        self.bn1 = nn.BatchNorm1d(n_filters_1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)

        self.conv2 = nn.Conv1d(n_filters_1, n_filters_2, kernel_size=7, padding=3)
        self.bn2 = nn.BatchNorm1d(n_filters_2)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2)

        self.conv3 = nn.Conv1d(n_filters_2, n_filters_3, kernel_size=7, padding=3)
        self.bn3 = nn.BatchNorm1d(n_filters_3)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool1d(kernel_size=2)
        
        with torch.no_grad():
            dummy_input_channel = 1
            dummy = torch.randn(1, dummy_input_channel, num_features_in)
            dummy = self.pool1(self.relu1(self.bn1(self.conv1(dummy))))
            dummy = self.pool2(self.relu2(self.bn2(self.conv2(dummy))))
            dummy = self.pool3(self.relu3(self.bn3(self.conv3(dummy))))
            flattened_size = dummy.shape[1] * dummy.shape[2]

        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(flattened_size, fc_units)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(fc_units, num_classes_out)

    def forward(self, x):
        x = self.pool1(self.relu1(self.bn1(self.conv1(x))))
        x = self.pool2(self.relu2(self.bn2(self.conv2(x))))
        x = self.pool3(self.relu3(self.bn3(self.conv3(x))))
        x = self.flatten(x); x = self.dropout(x)
        x = self.relu4(self.fc1(x)); x = self.fc2(x)
        return x

# Optuna Objective Function
def objective_cnn(trial):
    # Hyperparameters to tune
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)
    dropout_rate = trial.suggest_float("dropout_rate", 0.2, 0.7)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    
    n_filters_1 = trial.suggest_categorical("n_filters_1", [32, 64])
    n_filters_2 = trial.suggest_categorical("n_filters_2", [64, 128])
    n_filters_3 = trial.suggest_categorical("n_filters_3", [128, 256])
    fc_units = trial.suggest_categorical("fc_units", [128, 256, 512])
    
    label_smoothing = trial.suggest_float("label_smoothing", 0.0, 0.2) # Tunable label smoothing

    # Model, Optimizer, Criterion
    model = CNN1D(
        num_features_in=num_input_features_cnn_global,
        num_classes_out=num_classes_global,
        n_filters_1=n_filters_1,
        n_filters_2=n_filters_2,
        n_filters_3=n_filters_3,
        fc_units=fc_units,
        dropout_rate=dropout_rate
    ).to(device)

    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    # Handle class weights if y_train is imbalanced
    # Consider re-calculating unique_classes and y_train if they are not globally accessible or are modified
    # For simplicity, assuming y_train (original labels before encoding for CNN) is available for weight calculation
    # If class imbalance is a major issue, this should be done carefully
    # class_weights_array = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
    # class_weights_tensor = torch.tensor(class_weights_array, dtype=torch.float32).to(device)
    # criterion = nn.CrossEntropyLoss(weight=class_weights_tensor, label_smoothing=label_smoothing)
    criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing) # Simpler version without weights for now


    # DataLoaders
    train_dataset_opt = TensorDataset(X_train_cnn_opt, y_train_cnn_opt)
    val_dataset_opt = TensorDataset(X_val_cnn_opt, y_val_cnn_opt)
    train_loader_opt = DataLoader(train_dataset_opt, batch_size=batch_size, shuffle=True)
    val_loader_opt = DataLoader(val_dataset_opt, batch_size=batch_size, shuffle=False)

    # Training loop for Optuna (simplified for brevity, add your full loop with early stopping)
    num_epochs_optuna_trial = 50 # Number of epochs PER Optuna trial (can be less than final training)
    best_val_acc_trial = 0.0
    patience_optuna_trial = 10 # Early stopping patience for the trial
    patience_counter_trial = 0

    for epoch in range(num_epochs_optuna_trial):
        model.train()
        for inputs, labels in train_loader_opt:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        correct = 0
        total = 0
        val_loss_epoch = 0
        with torch.no_grad():
            for inputs, labels in val_loader_opt:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss_v = criterion(outputs, labels)
                val_loss_epoch += loss_v.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        epoch_val_acc = correct / total
        # epoch_val_loss_avg = val_loss_epoch / len(val_loader_opt.dataset) # Optional: for scheduler based on loss

        if epoch_val_acc > best_val_acc_trial:
            best_val_acc_trial = epoch_val_acc
            patience_counter_trial = 0
        else:
            patience_counter_trial += 1
        
        if patience_counter_trial >= patience_optuna_trial:
            print(f"Optuna Trial {trial.number}: Early stopping at epoch {epoch+1}")
            break
        
        # Optuna pruning: report intermediate value and check if trial should be pruned
        trial.report(epoch_val_acc, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
            
    return best_val_acc_trial # Optuna tries to maximize this value

# --- Run Optuna Study ---
print("\nStarting Optuna study for 1D-CNN hyperparameter tuning...")
# You can add a pruner for more efficient search, e.g., MedianPruner
study_cnn = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
study_cnn.optimize(objective_cnn, n_trials=30)  # Adjust n_trials (e.g., 20-100 depending on time)

print("Optuna study finished.")
print("Best CNN trial:")
best_trial_cnn = study_cnn.best_trial
print(f"  Value (Val Acc): {best_trial_cnn.value:.4f}")
print("  Params: ")
for key, value in best_trial_cnn.params.items():
    print(f"    {key}: {value}")

# --- Train final CNN model with best Optuna hyperparameters ---
print("\nTraining final 1D-CNN with best Optuna hyperparameters...")
best_cnn_params = best_trial_cnn.params

final_cnn_model = CNN1D(
    num_features_in=num_input_features_cnn_global,
    num_classes_out=num_classes_global,
    n_filters_1=best_cnn_params['n_filters_1'],
    n_filters_2=best_cnn_params['n_filters_2'],
    n_filters_3=best_cnn_params['n_filters_3'],
    fc_units=best_cnn_params['fc_units'],
    dropout_rate=best_cnn_params['dropout_rate']
).to(device)

final_optimizer = optim.AdamW(
    final_cnn_model.parameters(),
    lr=best_cnn_params['lr'],
    weight_decay=best_cnn_params['weight_decay']
)
# Re-calculate class weights if needed for the final model
# final_criterion = nn.CrossEntropyLoss(weight=class_weights_tensor, label_smoothing=best_cnn_params['label_smoothing'])
final_criterion = nn.CrossEntropyLoss(label_smoothing=best_cnn_params['label_smoothing'])


final_batch_size = best_cnn_params['batch_size']
train_dataset_final = TensorDataset(X_train_cnn_opt, y_train_cnn_opt) # Using _opt versions for consistency
val_dataset_final = TensorDataset(X_val_cnn_opt, y_val_cnn_opt)
train_loader_final = DataLoader(train_dataset_final, batch_size=final_batch_size, shuffle=True)
val_loader_final = DataLoader(val_dataset_final, batch_size=final_batch_size, shuffle=False)

# Use your full training loop here for the final model
num_epochs_cnn_final = 150 # Train for more epochs with best params
best_val_acc_cnn_final = 0.0
patience_cnn_final = 25    # Longer patience for final model
patience_counter_cnn_final = 0
scheduler_final = optim.lr_scheduler.ReduceLROnPlateau(final_optimizer, mode='min', patience=10, factor=0.5) # Sched for final train

for epoch in range(num_epochs_cnn_final):
    final_cnn_model.train()
    running_loss = 0.0
    for inputs, labels in train_loader_final:
        inputs, labels = inputs.to(device), labels.to(device)
        final_optimizer.zero_grad()
        outputs = final_cnn_model(inputs)
        loss = final_criterion(outputs, labels)
        loss.backward()
        final_optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader_final.dataset)

    final_cnn_model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader_final:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = final_cnn_model(inputs)
            loss_v = final_criterion(outputs, labels)
            val_loss += loss_v.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_val_loss = val_loss / len(val_loader_final.dataset)
    epoch_val_acc = correct / total
    print(f"Final CNN Train - Epoch {epoch+1}/{num_epochs_cnn_final}, Train Loss: {epoch_loss:.4f}, Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}")

    scheduler_final.step(epoch_val_loss)

    if epoch_val_acc > best_val_acc_cnn_final:
        best_val_acc_cnn_final = epoch_val_acc
        torch.save(final_cnn_model.state_dict(), 'best_tuned_cnn_1d_model.pth')
        print(f"New best tuned CNN val acc: {best_val_acc_cnn_final:.4f}. Model saved.")
        patience_counter_cnn_final = 0
    else:
        patience_counter_cnn_final += 1
    
    if patience_counter_cnn_final >= patience_cnn_final:
        print(f"Final CNN Early stopping at epoch {epoch+1}.")
        break

# Load the best tuned CNN model for evaluation and prediction
if os.path.exists('best_tuned_cnn_1d_model.pth'):
    final_cnn_model.load_state_dict(torch.load('best_tuned_cnn_1d_model.pth'))
    print(f"Loaded best tuned CNN model with val acc: {best_val_acc_cnn_final:.4f}")
else:
    print("Warning: Best tuned CNN model file not found. Using last state.")


models['1D-CNN (Raw TS)'] = final_cnn_model # Store the best tuned PyTorch model
val_accuracies['1D-CNN (Raw TS)'] = best_val_acc_cnn_final
print(f"Best Tuned 1D-CNN Validation Accuracy: {best_val_acc_cnn_final:.4f}")

# ... (rest of your script: final predictions, summary, recommendations)
# Ensure X_test_cnn_tensor is prepared for the final_cnn_model to predict on
X_test_cnn_tensor = torch.tensor(X_test_ts_scaled.reshape(-1, 1, X_test_ts_scaled.shape[1]), dtype=torch.float32)
test_dataset_cnn = TensorDataset(X_test_cnn_tensor)
test_loader_cnn = DataLoader(test_dataset_cnn, batch_size=final_batch_size, shuffle=False) # Use final batch size


Training XGBoost on engineered features (with EXPANDED GridSearchCV)...
Fitting 3 folds for each of 432 candidates, totalling 1296 fits


C:\Users\Semhane\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [00:29:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best XGBoost params: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 7, 'min_child_weight': 3, 'n_estimators': 300, 'subsample': 0.9}
Best XGBoost validation accuracy (from CV): 0.7069

Using CPU for PyTorch


[I 2025-05-17 00:29:23,313] A new study created in memory with name: no-name-df0eec55-3ad2-4058-bc4a-402e67b538a0



Starting Optuna study for 1D-CNN hyperparameter tuning...


...

[I 2025-05-17 00:53:35,437] Trial 23 finished with value: 0.73 and parameters: {'lr': 0.00037292174394559845, 'weight_decay': 0.0003079671550362641, 'dropout_rate': 0.6003673374746339, 'batch_size': 64, 'n_filters_1': 64, 'n_filters_2': 128, 'n_filters_3': 128, 'fc_units': 256, 'label_smoothing': 0.05462582690196247}. Best is trial 2 with value: 0.748.


Optuna Trial 23: Early stopping at epoch 19


[I 2025-05-17 00:53:50,729] Trial 24 pruned. 
[I 2025-05-17 00:54:35,997] Trial 25 pruned. 
[I 2025-05-17 00:54:42,984] Trial 26 pruned. 
[I 2025-05-17 00:54:47,471] Trial 27 pruned. 
[I 2025-05-17 00:54:50,847] Trial 28 pruned. 
[I 2025-05-17 00:57:07,900] Trial 29 finished with value: 0.73 and parameters: {'lr': 0.0007708825774272593, 'weight_decay': 8.479111953364705e-05, 'dropout_rate': 0.5575818082179422, 'batch_size': 32, 'n_filters_1': 64, 'n_filters_2': 128, 'n_filters_3': 128, 'fc_units': 128, 'label_smoothing': 0.08099484542704519}. Best is trial 2 with value: 0.748.


Optuna Trial 29: Early stopping at epoch 30
Optuna study finished.
Best CNN trial:
  Value (Val Acc): 0.7480
  Params: 
    lr: 0.001625383638022807
    weight_decay: 0.002978120991290357
    dropout_rate: 0.6701259674364999
    batch_size: 64
    n_filters_1: 32
    n_filters_2: 128
    n_filters_3: 128
    fc_units: 256
    label_smoothing: 0.1627328882060397

Training final 1D-CNN with best Optuna hyperparameters...
Final CNN Train - Epoch 1/150, Train Loss: 1.3502, Val Loss: 1.1871, Val Acc: 0.6120
New best tuned CNN val acc: 0.6120. Model saved.
Final CNN Train - Epoch 2/150, Train Loss: 1.1865, Val Loss: 1.1843, Val Acc: 0.5980
Final CNN Train - Epoch 3/150, Train Loss: 1.1706, Val Loss: 1.1370, Val Acc: 0.6170
New best tuned CNN val acc: 0.6170. Model saved.
Final CNN Train - Epoch 4/150, Train Loss: 1.0931, Val Loss: 1.0960, Val Acc: 0.6700
New best tuned CNN val acc: 0.6700. Model saved.
Final CNN Train - Epoch 5/150, Train Loss: 1.0768, Val Loss: 1.1585, Val Acc: 0.6160
Final

NameError: name 'os' is not defined

In [10]:
import os
# Load the best tuned CNN model for evaluation and prediction
if os.path.exists('best_tuned_cnn_1d_model.pth'):
    final_cnn_model.load_state_dict(torch.load('best_tuned_cnn_1d_model.pth'))
    print(f"Loaded best tuned CNN model with val acc: {best_val_acc_cnn_final:.4f}")
else:
    print("Warning: Best tuned CNN model file not found. Using last state.")


models['1D-CNN (Raw TS)'] = final_cnn_model # Store the best tuned PyTorch model
val_accuracies['1D-CNN (Raw TS)'] = best_val_acc_cnn_final
print(f"Best Tuned 1D-CNN Validation Accuracy: {best_val_acc_cnn_final:.4f}")

# ... (rest of your script: final predictions, summary, recommendations)
# Ensure X_test_cnn_tensor is prepared for the final_cnn_model to predict on
X_test_cnn_tensor = torch.tensor(X_test_ts_scaled.reshape(-1, 1, X_test_ts_scaled.shape[1]), dtype=torch.float32)
test_dataset_cnn = TensorDataset(X_test_cnn_tensor)
test_loader_cnn = DataLoader(test_dataset_cnn, batch_size=final_batch_size, shuffle=False) # Use final batch size

Loaded best tuned CNN model with val acc: 0.7500
Best Tuned 1D-CNN Validation Accuracy: 0.7500


## Objective ## 
This report details the hyperparameter optimization process and results for two key models in the sleep stage classification pipeline:
- **XGBoost Classifier** using engineered features, tuned with GridSearchCV.
- **1D Convolutional Neural Network (CNN)** using raw scaled time series, tuned with Optuna.
- The goal was to improve the baseline performance of these models by systematically searching for more optimal hyperparameter configurations.
## Methodology ##
### 1. XGBoost Hyperparameter Tuning with GridSearchCV ###
- **Model:** XGBoost Classifier.
- **Input Features:** Scaled engineered features (Statistical, FFT, Wavelet - 92 features).
- **Tuning Tool:** sklearn.model_selection.GridSearchCV.
- **Cross-Validation**: StratifiedKFold with 3 splits, ensuring class proportions were maintained in each fold. The random_state was set for reproducibility.
Scoring Metric: accuracy.
- **Parameter Grid Searched:**
 - n_estimators: [200, 300, 400] (Number of boosting rounds)
 - learning_rate: [0.03, 0.05, 0.1] (Step size shrinkage)
 - max_depth: [5, 7, 9] (Maximum tree depth)
 - subsample: [0.8, 0.9] (Fraction of samples used per tree)
 - colsample_bytree: [0.8, 0.9] (Fraction of features used per tree)
 - gamma: [0, 0.1] (Minimum loss reduction for a split)
 - min_child_weight: [1, 3] (Minimum sum of instance weight needed in a child)
- **Execution:** A total of 432 hyperparameter combinations were evaluated (3 folds each, resulting in 1296 fits).
### 2. 1D-CNN Hyperparameter Tuning with Optuna ###
- **Model:** Custom 1D Convolutional Neural Network (PyTorch).
- **Architecture:** 3 Conv1D layers (kernel=7, padding=3, BatchNorm, ReLU, MaxPool1D) followed by Flatten, Dropout, and 2 Dense layers. The number of filters in convolutional layers and units in the dense layer were part of the tuning.
- **Input Features:** Raw scaled time series data.
- **Tuning Tool:** Optuna library.
- **Study Direction:** Maximize validation accuracy.
- **Pruner:** MedianPruner (to stop unpromising trials early).
Number of Trials: 30.
- **Evaluation per Trial:** Each trial trained the CNN for up to 50 epochs with early stopping (patience=10) on the validation set.
- **Hyperparameters Searched by Optuna:**
 - lr (learning rate): Log-uniform between 1e-4 and 1e-2.
 - weight_decay: Log-uniform between 1e-5 and 1e-2.
 - dropout_rate: Uniform between 0.2 and 0.7.
 - batch_size: Categorical from [32, 64, 128].
 - n_filters_1, n_filters_2, n_filters_3 (number of filters in conv layers):  Categorical (e.g., [32, 64], [64, 128], [128, 256]).
 - fc_units (units in the first dense layer): Categorical (e.g., [128, 256, 512]).
 - label_smoothing: Uniform between 0.0 and 0.2.
- **Final Model Training:** After Optuna identified the best hyperparameters, the CNN was re-trained using these parameters for a potentially longer duration (150 epochs) with more patience (25 epochs) for early stopping, and ReduceLROnPlateau learning rate scheduler.
## Results of Hyperparameter Optimization ##
#### 1. Tuned XGBoost (GridSearchCV) ####
- **Best Hyperparameters Found:**
 - colsample_bytree: 0.8
 - gamma: 0
 - learning_rate: 0.05
 - max_depth: 7
 - min_child_weight: 3
 - n_estimators: 300
 - subsample: 0.9
 - Best Cross-Validated Accuracy (from GridSearchCV): 0.7069
#### 2. Tuned 1D-CNN (Optuna + Final Training) ####
- **Best Hyperparameters Found by Optuna (Trial 2):**
 - lr: 0.001625
 - weight_decay: 0.002978
 - dropout_rate: 0.6701
 - batch_size: 64
 - n_filters_1: 32
 - n_filters_2: 128
 - n_filters_3: 128
 - fc_units: 256
 - label_smoothing: 0.1627
 - **Validation Accuracy of Best Optuna Trial (short training):** 0.7480
 - **Final Tuned 1D-CNN Validation Accuracy (after longer retraining with best params):** 0.7500
   - This retraining process with increased epochs and patience allowed the model to converge to a slightly better performance. The model was saved at epoch 39.
## Discussion & Conclusions ##
- **Impact of Tuning:**
 - **XGBoost:** The GridSearchCV identified parameters that yielded a cross-validated accuracy of approximately 0.707. This provides a robust estimate of the tuned XGBoost model's performance on unseen data within the training distribution. Compare this to the initial XGBoost result (0.7010) to see if tuning provided a significant lift with this specific grid.
 - **1D-CNN:** Optuna successfully guided the search for a high-performing CNN architecture and training configuration. The best Optuna trial achieved 0.7480 accuracy. Subsequent retraining with these optimal parameters for more epochs (with early stopping) further improved the validation accuracy to 0.7500. This highlights the benefit of a two-stage process: efficient search with Optuna followed by focused training.
- **Model Comparison Post-Tuning:** After hyperparameter optimization, the 1D-CNN (0.7500) remains the top-performing model on the validation set, followed by the tuned XGBoost (0.7069).
- **Observations from CNN Tuning:**
 - The Optuna study explored a range of architectural (filter counts, FC units) and regularization (dropout, weight decay, label smoothing) parameters. The best-found parameters suggest that a moderately complex CNN with significant dropout and label smoothing performs well.
 - The learning rate and weight decay values found are typical for AdamW optimization.
- **Limitations:**
 - The GridSearchCV for XGBoost was extensive but still a subset of all possible combinations.
 - The Optuna study for the CNN was limited to 30 trials; more trials could potentially uncover even better configurations.
 - The feature set for XGBoost did not include the computationally intensive ARIMA/Prophet features, which might offer further improvements if incorporated.
## Next Steps ##
Based on these optimization results:
- The Tuned 1D-CNN should be considered the primary candidate for final predictions.
-  Tuned XGBoost serves as a strong secondary model.
- Further improvements towards the higher accuracy target should focus on:
 - **Ensemble methods:** Combining predictions from the tuned CNN and tuned XGBoost (e.g., weighted averaging or stacking).
 - **Advanced Feature Engineering/Selection:** Especially for the XGBoost model.
 - **Data Augmentation:** For the CNN to potentially improve its robustness and generalization.
 - **Deeper Error Analysis:** Investigating misclassifications of the tuned CNN.